This notebook utilizes the Workbench and Client classes, which allows for convenient, notebook-based use of the sdk.

In [1]:
from datetime import time
from whendo.sdk.client import Client
from whendo.sdk.workbench import Workbench
from whendo.core.util import IP, PP, FilePathe, Dirs, TimeUnit
import whendo.core.actions.file_action as file_x
import whendo.core.actions.logic_action as logic_x
import whendo.core.scheduler as sched_x
import whendo.core.actions.gpio_action as gpio_x

"""
define the actions
"""
green_on = gpio_x.SetPin(pin=27, on=True)
green_off = gpio_x.SetPin(pin=27, on=False)
green_toggle = gpio_x.TogglePin(pin=27)
red_on = gpio_x.SetPin(pin=25, on=True)
red_off = gpio_x.SetPin(pin=25, on=False)
red_toggle = gpio_x.TogglePin(pin=25)
gpio_clear = gpio_x.Cleanup()
toggle_toggle = logic_x.ListAction(op_mode=logic_x.ListOpMode.ALL, action_list=[green_toggle, red_toggle])
file_heartbeat = file_x.FileHeartbeat(file="gpio_beat.txt")

"""
define the schedulers
"""
morning, evening = time(6,0,0), time(18,0,0)
daily_often = sched_x.TimelyScheduler(start=morning, stop=evening, interval=1)
nightly_often = sched_x.TimelyScheduler(start=evening, stop=morning, interval=1)
randomly_often = sched_x.RandomlyScheduler(time_unit=TimeUnit.second, low=2, high=5)
timely_at_00_sec = sched_x.TimelyScheduler(interval=1, second=00)
timely_at_30_sec = sched_x.TimelyScheduler(interval=1, second=30)


def add_actions(client):
    """
    add actions to the 'client' dispatcher
    """
    [ client.add_action(*action) for action in [
        ('green_on', green_on),
        ('green_off', green_off),
        ('green_toggle', green_toggle),
        ('red_on', red_on),
        ('red_off', red_off),
        ('red_toggle', red_toggle),
        ('gpio_clear', gpio_clear),
        ('toggle_toggle', toggle_toggle),
        ('file_heartbeat', file_heartbeat)
        ]
    ]

def add_schedulers(client):
    """
    add schedulers to the 'client' dispatcher
    """
    [ client.add_scheduler(*scheduler) for scheduler in [
        ('daily_often', daily_often),
        ('nightly_often', nightly_often),
        ('randomly_often', randomly_often),
        ('timely_at_00_sec', timely_at_00_sec),
        ('timely_at_30_sec', timely_at_30_sec)
        ]
    ]

def schedule_actions(client):
    """
    schedule the actions of interest
    """
    [ client.schedule_action(*stuff) for stuff in [
        ('randomly_often', 'red_toggle'),
        ('daily_often', 'green_toggle'),
        ('nightly_often', 'green_toggle'),
        ('randomly_often', 'file_heartbeat')
        # ('nightly_often', 'toggle_toggle')
        ]
    ]

def unschedule_schedulers(client):
    """
    unschedule the schedulers
    """
    [ client.unschedule_scheduler(scheduler) for scheduler in [ 
        'randomly_often', 'daily_often', 'nightly_often'
        ]
    ]
def all_on(client):
    client.execute_action('green_on')
    client.execute_action('red_on')

def all_off(client):
    client.execute_action('green_off')
    client.execute_action('red_off')

"""
set up workbench
"""
workbench = Workbench()
workbench.add_client('local', Client(host=IP.local))
workbench.add_client('pi4', Client(host='192.168.0.45'))
workbench.add_client('pi3', Client(host='192.168.0.46'))
local = workbench.get_client('local')
pi3 = workbench.get_client('pi3')
pi4 = workbench.get_client('pi4')

"""
convenience functions applied to a list of Clients
"""
def run(client_list):
    for c in client_list:
        c.clear_dispatcher()
        add_actions(c)
        add_schedulers(c)
        schedule_actions(c)
        c.run_jobs()
def stop(client_list):
    for c in client_list:
        c.stop_jobs()
def pprint(client_list):
    for c in client_list:
        print(f"job count={c.job_count()} for ({c})")
        c.load_dispatcher().pprint()
def on(client_list):
    for c in client_list:
        all_on(c)
def off(client_list):
    for c in client_list:
        all_off(c)


In [ ]:
pprint([pi3, pi4])

In [ ]:
run([pi3, pi4])

In [2]:
stop([pi3, pi4])

In [ ]:
on([pi3, pi4])

In [3]:
off([pi3, pi4])